In [1]:
import cudf.pandas
cudf.pandas.install()

import pandas as pd

In [2]:
import cupy as cp

gpus = cp.cuda.runtime.getDeviceCount()
if gpus:
    print("Available GPUs:")
    for i in range(gpus):
        print(f"GPU {i}: {cp.cuda.runtime.getDeviceProperties(i)['name']}")
else:
    print("No GPU available.")


Available GPUs:
GPU 0: b'NVIDIA GeForce RTX 3060'


In [3]:
import os
import language_tool_python
from spellchecker import SpellChecker
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import spacy
from textblob import TextBlob
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask import delayed, compute
from spellchecker import SpellChecker
import spacy
from textblob import TextBlob
import language_tool_python
import dask.dataframe as dd

In [4]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
output_file_name = 'wildberries_reviews.csv.gz'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv.gz' не найден. Начинаю загрузку...


Downloading...
From (original): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe
From (redirected): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe&confirm=t&uuid=c3f35975-747d-4780-9dd0-42664764e114
To: /workspace/wildberries_reviews.csv.gz
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 164M/164M [00:03<00:00, 54.1MB/s]

Файл '/workspace/wildberries_reviews.csv.gz' успешно загружен.


In [5]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')

Модель ru_core_news_lg уже установлена.


In [6]:
import subprocess
from language_tool_python import LanguageTool
# Запуск команды
process = subprocess.Popen(
    ['find', '/', '-name', 'languagetool-server.jar'],
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
    text=True
)

# Чтение вывода
output, _ = process.communicate()

# # Удаление пробелов и пустых строк
# LANGUAGETOOL_DIR = [line.strip() for line in output.splitlines() if line.strip()][0]

# Функция для установки необходимых пакетов и загрузки LanguageTool, если еще не загружено
def setup_languagetool():
    if not os.path.isdir(output):
        print("Установка необходимых пакетов и загрузка LanguageTool...")
        os.system('apt update && apt install -y default-jre wget unzip')
        os.system('wget https://languagetool.org/download/LanguageTool-stable.zip && unzip -o LanguageTool-stable.zip')
    else:
        print("LanguageTool уже установлен.")
        
temp = LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1})
temp.close()

Unzipping /tmp/tmpnc_ys2oo.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


In [7]:
import threading
import queue
from tqdm import tqdm

NUM_TOOLS = 6
MAX_CHECK_THREADS = 10

def create_tool(tools_queue, pbar):
    tool = language_tool_python.LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1, 'maxCheckThreads': MAX_CHECK_THREADS})
    tools_queue.put(tool)
    pbar.update(1)

def correct_text(text, tool):
    if not isinstance(text, str):
        return text
    try:
        corrected_text = tool.correct(text)
        return corrected_text if corrected_text != text else text
    except Exception as e:
        print(f"Ошибка исправления текста: {e}")
        return text

def process_text(text_queue, results, progress_bar):
    while not text_queue.empty():
        idx, text, tool = text_queue.get()
        corrected_text = correct_text(text, tool)
        results.append((idx, corrected_text))
        text_queue.task_done()
        progress_bar.update(1)

def process_data(file_path, output_file_path, num_rows, num_tools, save_interval=1000):
    tools_queue = queue.Queue()
    threads = []
    with tqdm(total=num_tools, desc="Создание экземпляров LanguageTool") as pbar:
        for _ in range(num_tools):
            thread = threading.Thread(target=create_tool, args=(tools_queue, pbar))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

    tools = [tools_queue.get() for _ in range(num_tools)]
    df = pd.read_csv(file_path, compression='gzip', nrows=num_rows)

    text_queue = queue.Queue()
    results = []

    with tqdm(total=len(df), desc="Заполнение очереди строками") as pbar:
        for idx, row in df.iterrows():
            tool = tools[idx % len(tools)]
            text_queue.put((idx, row['review_full_text'], tool))
            pbar.update(1)

    with tqdm(total=text_queue.qsize(), desc="Обработка текста") as progress_bar:
        threads = []
        for _ in range(num_tools):
            thread = threading.Thread(target=process_text, args=(text_queue, results, progress_bar))
            thread.start()
            threads.append(thread)

        text_queue.join()
        for thread in threads:
            thread.join()

    results.sort(key=lambda x: x[0])
    corrected_texts = [text for _, text in results]
    df['corrected_text'] = corrected_texts

    temp_files = []

    for i in range(0, len(df), save_interval):
        batch_df = df.iloc[i:i+save_interval]
        temp_file = f"{output_file_path}_part_{i // save_interval}.csv.gz"
        batch_df.to_csv(temp_file, compression='gzip', index=False)
        temp_files.append(temp_file)
        print(f"Сохранены строки с {i} по {i + save_interval - 1}")

    # Объединение всех временных файлов в один
    combined_df = pd.concat([pd.read_csv(f, compression='gzip') for f in temp_files], ignore_index=True)
    combined_df.to_csv(output_file_path, compression='gzip', index=False)
    print(f"Объединенные результаты сохранены в {output_file_path}")

    # Удаление временных файлов
    for f in temp_files:
        os.remove(f)

test_cases = [(100000, NUM_TOOLS)]
for num_rows, num_tools in test_cases:
    process_data('wildberries_reviews.csv.gz', 'corrected_wildberries_reviews.csv.gz', num_rows=num_rows, num_tools=num_tools)

Обработка текста: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [02:19<00:00, 21.58it/s]


Сохранены строки с 0 по 999
Сохранены строки с 1000 по 1999
Сохранены строки с 2000 по 2999
Объединенные результаты сохранены в corrected_wildberries_reviews.csv.gz


In [8]:
!ls

corrected_wildberries_reviews.csv.gz  cufile.log  wildberries_reviews.csv.gz


In [9]:
t_df = pd.read_csv('corrected_wildberries_reviews.csv.gz', compression='gzip')
t_df

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_texts
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."
...,...,...,...,...,...,...,...
2995,3021,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...
2996,3022,Помог на несколько дней,3,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Помог на несколько дней
2997,3023,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ..."
2998,3024,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п..."


In [12]:
t_df[(t_df["review_full_text"] != t_df["corrected_texts"])]

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_texts
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
9,9,"При первой поездки в лес, барабан отломился ко...",1,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"При первой поездки в лес, барабан отломился ко..."
12,12,установили. сразу начала подозрительно хрустет...,2,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Установили. Сразу начала подозрительно хрустет...
14,14,Очень хорошая оебедка,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Очень хорошая лебедка
16,16,"Полный комплект,поставил взамен сдохшей,товаро...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Полный комплект, поставил взамен сдохшей, това..."
...,...,...,...,...,...,...,...
2993,3019,"Залил,пока течь держит.",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Залил, пока течь держит."
2995,3021,Реально работает!!! Ваз 2107 инжектор 2014 год...,5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,Реально работает!!! Ваз 2107 инжектор 2014 год...
2997,3023,"Челябинская фирма по производству авто химии,н...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Челябинская фирма по производству авто химии, ..."
2998,3024,"Упаковано хорошо,все похоже на качественный пр...",5,LAVR / Индикатор утечек системы охлаждения авто,/Автотовары/Автокосметика и автохимия/Герметик...,https://www.wildberries.ru/catalog/91102762/fe...,"Упаковано хорошо, все похоже на качественный п..."
